# Example usage of f5mkupy

##### 1) Get the encryption key on the BIG-IP

```bash
> f5mku -K
BHDLd0bbao1VlwpTk1sioQ==
```

##### 2) Create a http profile to encrypt a specific cookie

```sh
> tmsh create ltm profile http http_encrypted_cookie \
 encrypt-cookies add { CookieName } \
 encrypt-cookie-secret secret_encryption_key

> tmsh list ltm profile http http_encrypted_cookie
ltm profile http http_encrypted_cookie {
    app-service none
    encrypt-cookie-secret $M$bn$btwo4IWf6ZpYap4QWG8DsJqnB2xW9HLv1VOAmMeIa0U=
    encrypt-cookies { CookieName }
}
```

In [5]:
# time for some python code

# save f5mku key to a variable, we will re-use this along all examples
F5MKU_KEY='BHDLd0bbao1VlwpTk1sioQ=='

# import decryption function from f5mkupy
from f5mkupy import decrypt

# decrypt the encrypt-cookie-secret $M$bn$btwo4IWf6ZpYap4QWG8DsJqnB2xW9HLv1VOAmMeIa0U=
print(
    decrypt(
        ciphertext='$M$bn$btwo4IWf6ZpYap4QWG8DsJqnB2xW9HLv1VOAmMeIa0U=',
        f5mku=F5MKU_KEY
    )
)

secret_encryption_key




lets do the same on the command like:

```sh
> f5mkupy decrypt \
    -k 'BHDLd0bbao1VlwpTk1sioQ==' \
    '$M$bn$btwo4IWf6ZpYap4QWG8DsJqnB2xW9HLv1VOAmMeIa0U='
secret_encryption_key
```

Same result as with the python code.

##### 3) Time to encrypt a new secret key

In [7]:
# import encryption function from f5mkupy
from f5mkupy import encrypt

print(
    encrypt(
        plaintext='newSecretKey',
        f5mku=F5MKU_KEY
    )
)

$M$Nh$2JzppOwb8rT4bLOgk11AvQ==




We can perform the same encryption on the command line:

```sh
> f5mkupy encrypt \
 -k 'BHDLd0bbao1VlwpTk1sioQ==' \
 'newSecretKey'
$M$uT$YjZdBZVDt47J9BF12ogypg==
```

Lets use the newly created key to update the BIG-IP configuration:


```sh

> tmsh modify ltm profile http http_encrypted_cookie \
 encrypt-cookie-secret '$M$Nh$2JzppOwb8rT4bLOgk11AvQ=='


> tmsh list ltm profile http http_encrypted_cookie
ltm profile http http_encrypted_cookie {
    app-service none
    encrypt-cookie-secret $M$yy$zrZ+FDp3bHnUAQy6/et06Q==
    encrypt-cookies { CookieName }
}
```

You probably noticed that the key shown in the `tmsh list` command does not match the one we set in the previous `tmsh modify` command.
The BIG-IP decrypts the newly set key and then re-encrypts it using a different salt, which leads to this behavior.

Lets validate that the new key is actually matching the plaintext key we used (`newSecretKey`) earlier by decrypting it again:

```sh
> f5mkupy decrypt \
    -k 'BHDLd0bbao1VlwpTk1sioQ==' \
    '$M$yy$zrZ+FDp3bHnUAQy6/et06Q=='
newSecretKey
```

Match!



##### 3) Why do the ciphertexts not match when the plaintext is the same? (Salt!)


As briefly mentioned, the ciphertexts change when it got added to the BIG-IP configuaration because it was decrypted and then re-encrypted with a new salt.


We can extract the salt from the ciphertext:


```sh
f5mkupy extract_salt '$M$Nh$2JzppOwb8rT4bLOgk11AvQ=='
Nh

f5mkupy extract_salt '$M$yy$zrZ+FDp3bHnUAQy6/et06Q=='
yy
```


Re-using the salt from the existing ciphertext we can generate the exact _same_ ciphertext using the exact same plaintext.


This allows us to check if the plaintext is the one we expect it to be, which is great to implement idempotency! :)


```sh
f5mkupy encrypt -k 'BHDLd0bbao1VlwpTk1sioQ==' -s 'Nh' 'newSecretKey'
$M$Nh$2JzppOwb8rT4bLOgk11AvQ==

f5mkupy encrypt -k 'BHDLd0bbao1VlwpTk1sioQ==' -s 'yy' 'newSecretKey'
$M$yy$zrZ+FDp3bHnUAQy6/et06Q==
```


This gets a little easier with python:

In [26]:
# import extract_salt function from f5mkupy
from f5mkupy import extract_salt

# the ciphertexts from the above example:
ciphertexts=('$M$Nh$2JzppOwb8rT4bLOgk11AvQ==', '$M$yy$zrZ+FDp3bHnUAQy6/et06Q==')

# loop through the ciphertexts, extract salt and encrypt the same plaintext
for orig_ciphertext in ciphertexts:
    salt = extract_salt(orig_ciphertext)
    new_ciphertext = encrypt(plaintext='newSecretKey', f5mku=F5MKU_KEY, salt=salt)
    print(
        f"\nWhen encrypting the same plaintext with the same f5mku key and salt:'{salt}' the ciphertexts " +
       ( orig_ciphertext==new_ciphertext and "match" or "do not match" )
    )


When encrypting the same plaintext with the same f5mku key and salt:'Nh' the ciphertexts match

When encrypting the same plaintext with the same f5mku key and salt:'yy' the ciphertexts match


The end.